If you're not running in Saturn Cloud, you need to install these libraries:

Make sure you use the latest versions

```
pip install -U transformers accelerate bitsandbytes
```

In [1]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
none            3.8G  4.0K  3.8G   1% /mnt/wsl
drivers         472G  228G  244G  49% /usr/lib/wsl/drivers
none            3.8G     0  3.8G   0% /usr/lib/modules
none            3.8G     0  3.8G   0% /usr/lib/modules/5.15.153.1-microsoft-standard-WSL2
/dev/sdc       1007G   57G  899G   6% /
none            3.8G   96K  3.8G   1% /mnt/wslg
none            3.8G     0  3.8G   0% /usr/lib/wsl/lib
rootfs          3.8G  2.1M  3.8G   1% /init
none            3.8G  864K  3.8G   1% /run
none            3.8G     0  3.8G   0% /run/lock
none            3.8G  8.0K  3.8G   1% /run/shm
tmpfs           4.0M     0  4.0M   0% /sys/fs/cgroup
none            3.8G   76K  3.8G   1% /mnt/wslg/versions.txt
none            3.8G   76K  3.8G   1% /mnt/wslg/doc
C:\             472G  228G  244G  49% /mnt/c
snapfuse        128K  128K     0 100% /snap/bare/5
snapfuse         75M   75M     0 100% /snap/core22/1612
snapfuse         75M   75M     0 100% /snap/core22/1586
s

This line is here to put the files in a certain location where there is enough space to run the model.

In [2]:
import os
os.environ['HF_HOME'] = '/mnt/c/Users/Bruno/Documents/models'

In [3]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-09-16 15:54:55--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0.001s  

2024-09-16 15:54:56 (3.08 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [4]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [5]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [6]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [7]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
none            3.8G  4.0K  3.8G   1% /mnt/wsl
drivers         472G  228G  244G  49% /usr/lib/wsl/drivers
none            3.8G     0  3.8G   0% /usr/lib/modules
none            3.8G     0  3.8G   0% /usr/lib/modules/5.15.153.1-microsoft-standard-WSL2
/dev/sdc       1007G   57G  899G   6% /
none            3.8G   96K  3.8G   1% /mnt/wslg
none            3.8G     0  3.8G   0% /usr/lib/wsl/lib
rootfs          3.8G  2.1M  3.8G   1% /init
none            3.8G  864K  3.8G   1% /run
none            3.8G     0  3.8G   0% /run/lock
none            3.8G  8.0K  3.8G   1% /run/shm
tmpfs           4.0M     0  4.0M   0% /sys/fs/cgroup
none            3.8G   76K  3.8G   1% /mnt/wslg/versions.txt
none            3.8G   76K  3.8G   1% /mnt/wslg/doc
C:\             472G  228G  244G  49% /mnt/c
snapfuse        128K  128K     0 100% /snap/bare/5
snapfuse         75M   75M     0 100% /snap/core22/1612
snapfuse         75M   75M     0 100% /snap/core22/1586
s

In [8]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [9]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/home/bruno/miniconda3/envs/llm-zoomcamp/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


Nu kun je de build prompt verbeteren met dit andere model, dat lokaal draait ipv een mistral server

In [10]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids, )
    result = tokenizer.decode(outputs[0])
    return result

In [11]:
def llm(prompt, generate_params=None):
    if generate_params is None:
        generate_params = {}

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(
        input_ids,
        max_length=generate_params.get("max_length", 100),
        num_beams=generate_params.get("num_beams", 5),
        do_sample=generate_params.get("do_sample", False),
        temperature=generate_params.get("temperature", 1.0),
        top_k=generate_params.get("top_k", 50),
        top_p=generate_params.get("top_p", 0.95),
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result


In [12]:
rag("I just discovered the course. Can I still join it?")

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx